In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [3]:
# 우리가 데이터를 가져올 웹 페이지입니다.
faker_opgg_url = 'https://www.op.gg/summoner/userName=Hide%20on%20bush'

# selenium이 제어할 chrome을 실행합니다.
driver = webdriver.Chrome("C:/Users/LUNA/Desktop/OPGG/chromedriver_win32/chromedriver")
# driver = webdriver.Chrome("C:/Users/LUNA/Desktop/OPGG/chromedriver_win32/chromedriver", options= options)

In [4]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(faker_opgg_url)

In [5]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

In [6]:
# 게임이 모두 로딩된 페이지의 html 코드를 faker_total_html로 저장합니다.
faker_total_html = driver.page_source

# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [7]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
faker_total_soup = BeautifulSoup(faker_total_html, 'html.parser')

In [8]:
# 결과가 들어갈 빈 리스트를 만듭니다.
faker_champions = []
faker_kills = []
faker_deaths = []
faker_assists = []
faker_results = []

In [9]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
faker_total_games_html = faker_total_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(faker_total_games_html)
print(total_game_len)

707


In [10]:
# 각 게임에 대해 웹 페이지에 기재된 스탯을 찾아서(selector 사용) 결과 리스트에 append하기
for i in range(total_game_len):
    faker_champions.append(list(faker_total_games_html[i].select_one('div.ChampionName').
                                               stripped_strings)[0])
    faker_kills.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Kill').
                                               stripped_strings)[0])
    faker_deaths.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Death').
                                               stripped_strings)[0])
    faker_assists.append(list(faker_total_games_html[i].select_one('div.KDA div.KDA span.Assist').
                                               stripped_strings)[0])
    faker_results.append(list(faker_total_games_html[i].select_one('div.GameStats div.GameResult').
                                               stripped_strings)[0])

In [11]:
# DataFrame으로 변환 후 출력
faker_total_df = pd.DataFrame([faker_champions,
                                faker_results,
                                faker_kills,
                                faker_deaths,
                                faker_assists],
                               index = ['champion', 'result', 'kills', 'deaths', 'assists']).T
faker_total_df

,champion,result,kills,deaths,assists
0,이렐리아,패배,3,6,4
1,이렐리아,승리,9,2,8
2,레오나,패배,4,3,19
3,세라핀,패배,0,3,2
4,이렐리아,승리,4,0,5
...,...,...,...,...,...
702,키아나,승리,14,6,8
703,키아나,승리,17,3,4
704,신드라,승리,3,4,6
705,신드라,패배,2,5,2


In [12]:
faker_total_df2 = faker_total_df.copy()

In [13]:
faker_total_df2['champion'].value_counts()

아칼리     77
루시안     65
사일러스    61
이렐리아    53
르블랑     39
        ..
잔나       1
세주아니     1
헤카림      1
말자하      1
피들스틱     1
Name: champion, Length: 83, dtype: int64